# HiFi-GAN Vocoder Demo

This notebook demonstrates the HiFi-GAN vocoder trained on RUSLAN dataset for Russian TTS.

**Paper:** [HiFi-GAN: Generative Adversarial Networks for Efficient and High Fidelity Speech Synthesis](https://arxiv.org/abs/2010.05646)

**Training Logs:** [CometML](https://www.comet.com/krugd/hifigan-russian-tts/444cc4e3c69c439cbd2f2186e108e03c)

## Contents
1. Setup and Installation
2. Download Pre-trained Weights
3. Download Test Audio
4. Load Model and Synthesize
5. Resynthesize Mode Demo
6. Batch Synthesis with synthesize.py
7. Full TTS Pipeline (Optional)
8. Performance Summary

## 1. Setup and Installation

In [ ]:
# Check environment
import sys
IN_COLAB = 'google.colab' in sys.modules
print(f"Running in Google Colab: {IN_COLAB}")

In [ ]:
# Clone repository (Colab only)
if IN_COLAB:
    !git clone https://github.com/KrugD/HW3_TTS.git
    %cd HW3_TTS
else:
    print("Running locally - skipping clone")

In [ ]:
# Install dependencies from requirements.txt
!pip install -q -r requirements.txt

In [ ]:
# Setup paths
import os
import sys
from pathlib import Path

if IN_COLAB:
    PROJECT_ROOT = Path('/content/HW3_TTS')
else:
    PROJECT_ROOT = Path('.').absolute()

sys.path.insert(0, str(PROJECT_ROOT))
os.chdir(PROJECT_ROOT)

print(f"Project root: {PROJECT_ROOT}")
print(f"Working directory: {os.getcwd()}")

In [ ]:
import time
import urllib.request
import zipfile

import gdown
import numpy as np
import pandas as pd
import torch
import torchaudio
import matplotlib.pyplot as plt
from IPython.display import Audio, display

# Import project modules (requires PROJECT_ROOT in sys.path)
from src.model import Generator
from src.transforms import MelSpectrogram, MelSpectrogramConfig

print("All imports loaded successfully!")

## 2. Download Pre-trained Weights

In [ ]:
# Create directories
WEIGHTS_DIR = PROJECT_ROOT / 'models_weights'
TEST_DIR = PROJECT_ROOT / 'data' / 'test' / 'audio'
OUTPUT_DIR = PROJECT_ROOT / 'output' / 'synthesized'

WEIGHTS_DIR.mkdir(parents=True, exist_ok=True)
TEST_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Download weights from Google Drive
WEIGHTS_PATH = WEIGHTS_DIR / 'generator_best.pt'
WEIGHTS_FILE_ID = "1LTl3m5ZScfefGvQUXTBAYfBeoSKhDZR2"

if not WEIGHTS_PATH.exists():
    print("Downloading HiFi-GAN generator weights...")
    gdown.download(
        f"https://drive.google.com/uc?id={WEIGHTS_FILE_ID}",
        str(WEIGHTS_PATH),
        quiet=False
    )
    print(f"Downloaded to: {WEIGHTS_PATH}")
else:
    print(f"Weights already exist: {WEIGHTS_PATH}")

# Verify file size
if WEIGHTS_PATH.exists():
    size_mb = WEIGHTS_PATH.stat().st_size / (1024 * 1024)
    print(f"File size: {size_mb:.2f} MB")

## 3. Download Test Audio from RUSLAN Dataset

In [ ]:
# Download a few samples from RUSLAN for testing
# RUSLAN dataset: https://ruslan-corpus.github.io/

# We'll download a small subset for demo purposes
# You can also provide your own .wav files in data/test/audio/

RUSLAN_SAMPLE_URL = "https://ruslan-corpus.github.io/data/ruslan_sample.zip"

# Check if we already have test files
existing_files = list(TEST_DIR.glob('*.wav'))
print(f"Found {len(existing_files)} existing test audio files")

if len(existing_files) == 0:
    print("\nNo test audio found. You can:")
    print("1. Upload your own .wav files to data/test/audio/")
    print("2. Download RUSLAN dataset with: python download_ruslan.py")
    print("\nFor this demo, we'll create a simple test audio.")

In [ ]:
# If no test files, allow upload or use synthetic audio for structure demo
if IN_COLAB and len(list(TEST_DIR.glob('*.wav'))) == 0:
    from google.colab import files
    print("Upload .wav files for testing (or skip to use synthetic audio):")
    try:
        uploaded = files.upload()
        for filename, content in uploaded.items():
            with open(TEST_DIR / filename, 'wb') as f:
                f.write(content)
            print(f"Saved: {filename}")
    except:
        print("Upload skipped or failed")

# Fallback: create synthetic test audio if needed
if len(list(TEST_DIR.glob('*.wav'))) == 0:
    print("\nCreating synthetic test audio for demo structure...")
    print("Note: For real quality testing, use actual speech recordings!")
    
    # Create a simple tone (just to show the pipeline works)
    sr = 22050
    duration = 2.0
    t = np.linspace(0, duration, int(sr * duration), dtype=np.float32)
    # Simple harmonic tone
    audio = 0.5 * np.sin(2 * np.pi * 220 * t) + 0.3 * np.sin(2 * np.pi * 440 * t)
    audio = audio * np.exp(-t * 0.5)  # Decay envelope
    audio = torch.from_numpy(audio).unsqueeze(0)
    torchaudio.save(str(TEST_DIR / 'test_tone.wav'), audio, sr)
    print(f"Created: {TEST_DIR / 'test_tone.wav'}")

## 4. Load Model and Synthesize

In [ ]:
# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

In [ ]:
# Create mel-spectrogram transform (same config as training)
mel_config = MelSpectrogramConfig(
    sr=22050,
    n_fft=1024,
    win_length=1024,
    hop_length=256,
    n_mels=80,
    f_min=0.0,
    f_max=8000.0,
)
mel_transform = MelSpectrogram(mel_config).to(device)
print("Mel-spectrogram transform created")
print(f"  Sample rate: {mel_config.sr} Hz")
print(f"  Mel bands: {mel_config.n_mels}")
print(f"  Hop length: {mel_config.hop_length}")

In [ ]:
# Load HiFi-GAN Generator V1
generator = Generator(
    in_channels=80,
    hidden_channels=512,
    upsample_rates=(8, 8, 2, 2),
    upsample_kernel_sizes=(16, 16, 4, 4),
    resblock_kernel_sizes=(3, 7, 11),
    resblock_dilations=((1, 3, 5), (1, 3, 5), (1, 3, 5)),
)

# Load weights
checkpoint = torch.load(WEIGHTS_PATH, map_location=device)
if isinstance(checkpoint, dict) and 'generator' in checkpoint:
    generator.load_state_dict(checkpoint['generator'])
else:
    generator.load_state_dict(checkpoint)

generator = generator.to(device)
generator.eval()
generator.remove_weight_norm()  # Faster inference

# Count parameters
n_params = sum(p.numel() for p in generator.parameters())
print(f"\nGenerator loaded successfully!")
print(f"Parameters: {n_params:,} (~{n_params/1e6:.1f}M)")

## 5. Resynthesize Mode Demo

In this mode, we:
1. Load ground-truth audio
2. Extract mel-spectrogram
3. Generate audio from mel-spectrogram
4. Compare original vs synthesized

In [ ]:
@torch.no_grad()
def resynthesize(audio_path, generator, mel_transform, device):
    """
    Resynthesize audio: load -> mel -> vocoder -> audio
    
    Returns:
        original audio, generated audio, mel spectrogram, synthesis time
    """
    # Load and preprocess audio
    audio, sr = torchaudio.load(audio_path)
    if audio.shape[0] > 1:
        audio = audio.mean(dim=0, keepdim=True)
    audio = audio.squeeze(0)
    
    # Resample if needed
    if sr != mel_config.sr:
        resampler = torchaudio.transforms.Resample(sr, mel_config.sr)
        audio = resampler(audio)
    
    # Normalize
    if audio.abs().max() > 0:
        audio = audio / audio.abs().max()
    
    # Extract mel-spectrogram
    mel = mel_transform(audio.to(device))
    
    # Synthesize
    start_time = time.time()
    audio_gen = generator(mel.unsqueeze(0))
    synthesis_time = time.time() - start_time
    
    audio_gen = audio_gen.squeeze().cpu()
    
    # Normalize output
    if audio_gen.abs().max() > 0:
        audio_gen = audio_gen / audio_gen.abs().max() * 0.95
    
    return audio.cpu(), audio_gen, mel.cpu(), synthesis_time


def plot_comparison(audio_orig, audio_gen, mel, sr=22050):
    """Plot waveforms and spectrogram comparison."""
    fig, axes = plt.subplots(3, 1, figsize=(14, 8))
    
    # Time axis
    t_orig = torch.arange(len(audio_orig)) / sr
    t_gen = torch.arange(len(audio_gen)) / sr
    
    # Original waveform
    axes[0].plot(t_orig.numpy(), audio_orig.numpy(), linewidth=0.5)
    axes[0].set_title('Original Audio Waveform')
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude')
    axes[0].set_xlim([0, t_orig[-1]])
    axes[0].grid(True, alpha=0.3)
    
    # Generated waveform
    axes[1].plot(t_gen.numpy(), audio_gen.numpy(), linewidth=0.5, color='orange')
    axes[1].set_title('Generated Audio Waveform (HiFi-GAN)')
    axes[1].set_xlabel('Time (s)')
    axes[1].set_ylabel('Amplitude')
    axes[1].set_xlim([0, t_gen[-1]])
    axes[1].grid(True, alpha=0.3)
    
    # Mel-spectrogram
    im = axes[2].imshow(mel.numpy(), aspect='auto', origin='lower', cmap='viridis')
    axes[2].set_title('Mel-Spectrogram (Input to Vocoder)')
    axes[2].set_xlabel('Time Frames')
    axes[2].set_ylabel('Mel Frequency Bin')
    plt.colorbar(im, ax=axes[2], label='Log Magnitude')
    
    plt.tight_layout()
    plt.show()
    return fig

In [ ]:
# Process all test audio files
audio_files = sorted(TEST_DIR.glob('*.wav'))
print(f"Found {len(audio_files)} test audio files\n")

results = []

for audio_path in audio_files:
    print("=" * 60)
    print(f"File: {audio_path.name}")
    print("=" * 60)
    
    # Resynthesize
    audio_orig, audio_gen, mel, synth_time = resynthesize(
        audio_path, generator, mel_transform, device
    )
    
    # Calculate metrics
    audio_duration = len(audio_gen) / mel_config.sr
    rtf = synth_time / audio_duration
    speed = 1 / rtf
    
    print(f"Audio duration: {audio_duration:.2f} s")
    print(f"Synthesis time: {synth_time*1000:.1f} ms")
    print(f"Real-time factor: {rtf:.4f}")
    print(f"Speed: {speed:.1f}x faster than real-time")
    
    results.append({
        'file': audio_path.name,
        'duration': audio_duration,
        'synth_time': synth_time,
        'rtf': rtf,
        'speed': speed
    })
    
    # Plot comparison
    plot_comparison(audio_orig, audio_gen, mel)
    
    # Play audio
    print("\nOriginal Audio:")
    display(Audio(audio_orig.numpy(), rate=mel_config.sr))
    
    print("\nGenerated Audio (HiFi-GAN):")
    display(Audio(audio_gen.numpy(), rate=mel_config.sr))
    
    # Save generated audio
    output_path = OUTPUT_DIR / f"{audio_path.stem}_generated.wav"
    torchaudio.save(str(output_path), audio_gen.unsqueeze(0), mel_config.sr)
    print(f"\nSaved to: {output_path}")
    print()

## 6. Batch Synthesis with synthesize.py

In [ ]:
# Demonstrate synthesize.py script usage
print("Running synthesize.py for batch processing...")
print()

!python synthesize.py \
    input_dir=data/test/audio \
    output_dir=output/batch_synthesized \
    checkpoint=models_weights/generator_best.pt \
    resynthesize=true \
    device={device}

In [ ]:
# List generated files
batch_output = PROJECT_ROOT / 'output' / 'batch_synthesized'
if batch_output.exists():
    print("Generated files:")
    for f in sorted(batch_output.glob('*.wav')):
        size = f.stat().st_size / 1024
        print(f"  {f.name} ({size:.1f} KB)")

## 8. Performance Summary

In [ ]:
# Summary statistics
if results:
    df = pd.DataFrame(results)
    print("Performance Summary")
    print("=" * 50)
    print(df.to_string(index=False))
    print()
    print(f"Average speed: {df['speed'].mean():.1f}x real-time")
    print(f"Average RTF: {df['rtf'].mean():.4f}")
else:
    print("No results to summarize")

In [ ]:
# Model info summary
print("\nModel Information")
print("=" * 50)
print(f"Model: HiFi-GAN V1")
print(f"Parameters: {n_params:,}")
print(f"Device: {device}")
print(f"Sample Rate: {mel_config.sr} Hz")
print(f"Mel Bands: {mel_config.n_mels}")
print(f"Hop Length: {mel_config.hop_length}")

## Conclusion

This demo showed:

1. **Setup**: Cloning repo and installing dependencies
2. **Model Loading**: Loading pre-trained HiFi-GAN V1 from Google Drive
3. **Resynthesize Mode**: Ground-truth audio -> mel -> vocoder -> audio
4. **Batch Processing**: Using `synthesize.py` script
5. **Full TTS**: Integration with acoustic models (conceptual)
6. **Performance**: Real-time factor and speed metrics

For more details, see:
- [README.md](https://github.com/KrugD/HW3_TTS?tab=readme-ov-file)
- [HiFi-GAN Paper](https://arxiv.org/abs/2010.05646)
- [CometML Training Logs](https://www.comet.com/krugd/hifigan-russian-tts)